In [8]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
import cvxpy as cp
from plotly.subplots import make_subplots
from croniter import croniter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
instruments = [
    'META',
    'TSLA',
    'NVDA',
    'AAPL',
    'EWY',
    'DXJ',
    'BRK-B',
    'SPY',
    'QQQ'
]

end_date = get_today()
start_date = pd.to_datetime(datetime(2022, 1, 3))
max_leverage = 1.3
vol_target = 0.2

In [30]:
portfolio = PortfolioStandard(
    capital=169336.48958837058,
    name='MVOPortfolio',        
    rebalance_iter=RebalancerIter('0 0 * * Fri', 2),    
    tc_model=TransactionCostFutu(), 
    systems=[        
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments,  strategy=[MeanVarianceOpt(gamma=10, hhi=0.2, lookback=60, confidence=1)], max_leverage=max_leverage, offset=60)
    ]
)

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize()
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

2024-08-26 00:47:17,886 - TradingSubSystemSingle [MVO - SMA1] - INFO - Generating position for strategy MVO - SMA1 between 2021-12-31 and 2024-08-25......
100%|██████████| 691/691 [00:04<00:00, 165.95it/s]
2024-08-26 00:47:22,073 - TradingSubSystemSingle [MVO - SMA1] - INFO - Volatility Target = 20.0% | Price Volatility = 25.5% | Last Scale Factor = 0.79


,Rebalanced Portfolio,Optimized Portfolio,^SPX,MVO - SMA1
Measure,,,,
Cumulative Return,2.086703,2.029772,1.174719,2.085631
Annualized Return,0.305558,0.294797,0.077397,0.303970
Annualized Volatility,0.229762,0.228643,0.180336,0.223541
Annualized Sharpe Ratio,1.183268,1.141994,0.242377,1.209094
Maximum Drawdown,-0.268356,-0.263019,-0.254251,-0.247532
